# Setting up

## Modules

In [1]:
import sys
sys.path.append("../modules")

In [2]:
from cleaning_merging import *

In [3]:
from datetime import datetime
#from pandas import DataFrame
from IPython.display import HTML
#from google.cloud.storage import Blob
#import datalab.storage as gcs_datalab

## Data gathering variables

In [4]:
date_dataset = datetime.strptime("2018-01-01", '%Y-%m-%d')
columns_targets = ['date_month', 'id_company', 'id_branch', 'is_sole_proprietor', 'has_relocated']
columns_features = ['date_month', 'id_company', 'id_branch',
                    'is_discontinued', 'financial_calamity_outcome', 'date_established', 'qty_employees', 
                    'year_qty_employees', 'id_company_creditproxy', 'score_payment_assessment', 
                    'amt_revenue', 'year_revenue', 'amt_consolidated_revenue', 'year_consolidated_revenue',
                    'amt_consolidated_operating_result', 'year_consolidated_operating_result', 
                    'perc_credit_limit_adjustment', 'color_credit_status', 'rat_pd', 'score_pd',
                    'has_increased_risk', 'is_sole_proprietor', 'code_SBI_2', 'code_SBI_1',
                    'qty_address_mutations_total', 'qty_address_mutations_month', 'has_name_change', 
                    'code_discontinuation', 'code_financial_calamity', 'qty_issued_credit_reports', 
                    'Associate', 'Authorized official', 'Board member', 'Chairman', 'Commissioner', 
                    'Director', 'Liquidator', 'Major', 'Managing clerk', 'Managing partner', 
                    'Member of the partnership', 'Miscellaneous', 'Owner', 'Secretary', 'Secretary/Treasurer', 
                    'Treasurer', 'Unknown', 'Vice President', 'amt_operating_result', 'code_legal_form', 
                    'date_financial_calamity_started', 'date_financial_calamity_stopped', 'date_start', 
                    'from_date_start', 'qty_stopped_names', 'qty_started_names', 'year_operating_result']    

### For a loop

In [5]:
dates = ["2018-01-01", "2017-01-01", "2016-01-01", "2015-01-01", "2014-01-01", "2013-01-01"]
list_date_dataset = [datetime.strptime(date, '%Y-%m-%d') for date in dates]

In [6]:
date_dataset

datetime.datetime(2018, 1, 1, 0, 0)

## Project constants

In [7]:
project = 'graydon-moving-indicator'
bucket_name = 'graydon-data'
dir_input_data = '01_input'
clean_merge = Cleaner_Merger(project, bucket_name, dir_input_data)

# Gathering data

## Features

In [8]:
df_features = clean_merge.get_features(date_dataset, columns_features)

Read 01_input/2017/modelling_2017-01-01_2017-01-31.CSV with 5000 rows
After removing sole proprietors there are 4411 rows are left
The number of rows so far by adding 01_input/2017/modelling_2017-01-01_2017-01-31.CSV : 4411
Read 01_input/2017/modelling_2017-02-01_2017-02-28.CSV with 5000 rows
After removing sole proprietors there are 4412 rows are left
The number of rows so far by adding 01_input/2017/modelling_2017-02-01_2017-02-28.CSV : 8823
Read 01_input/2017/modelling_2017-03-01_2017-03-31.CSV with 5000 rows
After removing sole proprietors there are 4414 rows are left
The number of rows so far by adding 01_input/2017/modelling_2017-03-01_2017-03-31.CSV : 13237
Read 01_input/2017/modelling_2017-04-01_2017-04-30.CSV with 5000 rows
After removing sole proprietors there are 4414 rows are left
The number of rows so far by adding 01_input/2017/modelling_2017-04-01_2017-04-30.CSV : 17651
Read 01_input/2017/modelling_2017-05-01_2017-05-31.CSV with 5000 rows
After removing sole proprietors 

In [10]:
df_features.head()

,date_month,id_company,id_branch,date_established,is_discontinued,code_discontinuation,code_financial_calamity,date_financial_calamity_started,date_financial_calamity_stopped,financial_calamity_outcome,...,from_date_start,qty_started_names,qty_stopped_names,has_name_change,total_changeof_board_members_,date_relocation_last,date_relocation_penultimate,years_relocation_last,years_relocation_penultimate,date_dataset
0,2017-01-01,3,10079408,1921-03-17,False,NaN,NaN,NaN,NaN,-1.0,...,NaN,0,0,False,0,2015-06-11,2014-10-02,1.560607,2.25056,2018-01-01
1,2017-01-01,5,10079416,1740-01-01,False,NaN,NaN,NaN,NaN,-1.0,...,NaN,0,0,False,0,NaT,NaT,NaN,NaN,2018-01-01
2,2017-01-01,6,10079424,1874-11-20,False,NaN,F,2013-01-29,NaN,-1.0,...,NaN,0,0,False,0,2001-06-15,NaT,15.548574,NaN,2018-01-01
3,2017-01-01,9,10079432,1897-05-01,False,NaN,NaN,NaN,NaN,-1.0,...,NaN,0,0,False,0,1997-03-20,NaT,19.786854,NaN,2018-01-01
4,2017-01-01,12,35,1924-08-01,False,NaN,NaN,NaN,NaN,-1.0,...,NaN,0,0,False,0,NaT,NaT,NaN,NaN,2018-01-01


### Target

In [11]:
df_target = clean_merge.get_targets(date_dataset, columns_targets)

KeyboardInterrupt: 

In [ ]:
date_month = date_dataset

df_months_combined = pd.DataFrame()  # The data frame which will contain all independent variables
    
# Get all months in range
df_date_months = pd.DataFrame(pd.date_range(date_month, periods=12, freq="M").tolist(),
                              columns=['date_month'])
df_date_months['date_month'] = df_date_months['date_month'].values.astype('datetime64[M]') # First day of month

# Get the file names of all required month files
#month_files = get_month_filenames(df_date_months, bucket, dir_data)

In [ ]:
month_files = [] # List of month files

df_date_months['year'] = df_date_months.date_month.dt.year
list_years = df_date_months['year'].unique()
# If there are multiple years, iterate through years  
for year in list_years:
    # Get the year's data file names
    dir_data_year = dir_data + '/' + str(year)
    list_blob = list(bucket.list_blobs(prefix=dir_data_year))
    # finding out which month files should be processed by looking which contain the first month date (YYYY-mm-01)
    df_year_months = df_date_months[df_date_months['year'] == year]['date_month']
    for blob in list_blob:
        for month in df_year_months:
            if (month.strftime("%Y-%m-%d") in blob.name) & ('CSV' in blob.name):
                month_files.append(blob.name)

In [ ]:
month_files

In [ ]:
# Cleaning, transforming and combining month files                
for month_file in month_files:
    with fs.open('graydon-data/' + month_file) as f:
        df_month = pd.read_csv(f, sep=';', usecols= columns_features, index_col=False)   
        df_month = df_month[(df_month['is_sole_proprietor'] == 0)] # & (one_month_df['is_discontinued'] == 0) 
        df_month.columns = (df_month.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', ''))
        df_months_combined = df_months_combined.append(df_month)
        print('The number of rows so far by adding ', month_file, ":", df_months_combined.shape[0])

df_months_combined['date_dataset'] = date_month

# Aggregating data to year
df_months_combined = df_months_combined.groupby(['date_dataset', 
                                                  'id_company', 
                                                  'id_branch']).agg({'has_relocated': 'max', 
                                                                     'date_month': 'max'})
df_months_combined = df_months_combined.rename(index=str, columns={"date_month": "date_month_last"})
df_months_combined = df_months_combined.reset_index()
df_months_combined['date_dataset'] = pd.to_datetime(df_months_combined['date_dataset'])
df_months_combined['id_company'] = df_months_combined['id_company'].astype(int)
df_months_combined['id_branch'] = df_months_combined['id_branch'].astype(int)

return(df_months_combined)